In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot

from sklearn.metrics import mean_squared_error
from math import sqrt

from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

## Getting the Data Set 

In [2]:
# Loading the market data set 

# Probably a less circuitous route to getting this done, my pd skills are rusty...

md = pd.read_csv('Tech_Eval/market_data.csv')
inter = md[['base_asset_id', 'price_open']]
mdbtc = inter[inter['base_asset_id'] == 'Bitcoin_BTC_BTC']
mdbtc_arr = mdbtc['price_open'].values

In [3]:
# Two variables determining how we carve up our data. 
split = len(mdbtc_arr) - 40
train_size = int(len(mdbtc_arr) * .50)

# Arriving at a train and test set
dataset = mdbtc_arr[:split]
train, test = dataset[:train_size], dataset[train_size:]

# Setting aside a small chunk of the data for a final evaluation. 
# holdout =  mdbtc_arr[split:]

## Univariate Predictive Models

### Autoregression

In [4]:
model = AR(mdbtc_arr)
model_fit = model.fit()
yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr))
print(yhat[0])

13179.25688155909


### Moving-averages

In [5]:
model = ARMA(mdbtc_arr, order=(0, 1))
model_fit = model.fit(disp=False)

yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr))
print(yhat[0])

10466.667170191871


### Autoregressive Moving-averages

In [6]:
model = ARMA(mdbtc_arr, order=(2, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr))
print(yhat[0])

13543.738537098749


### Autoregressive Integrated Moving-averages (with validation)

All predictive analysis is predicated on making certain assumptions about the underlying data, and our success at reducing the uncertainty of the future depends in large part on how well these assumptions hold up. 

With time series data we are making the assumption that our data are stationary -- that is, that they do not have some underlying temporal structure which could be described by a [unit root test](https://en.wikipedia.org/wiki/Unit_root_test).

Below we have code adapted from [Jason Brownlee](https://machinelearningmastery.com/time-series-forecast-case-study-python-monthly-armed-robberies-boston/)

In [7]:
# Are we stationary? 

def difference(data):
    diff = list()
    for i in range(1, len(data)):
        value = data[i] - data[i - 1]
        diff.append(value)
    return pd.Series(diff)

stationary = difference(mdbtc_arr)
stationary.index = mdbtc.index[1:]

result = adfuller(stationary)
print('ADF Statistic: %f' % result[0])

ADF Statistic: -10.701278


A lower ADF score means a higher likelihood that the data are stationary and our analysis will be valid. 

##### Naive ARIMA, just getting used to the `statsmodel` interface and getting a basic iteration up and running.

In [8]:
model = ARIMA(dataset, order=(1, 2, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr), typ='levels')
print(yhat[0])

8537.84207485588


##### More sophisticated model, with validation 

Jason Brownlee of [Machine Learning Mastery](https://machinelearningmastery.com/) fame notes that there are two parts to validating a time series model - performance measure and testing strategy.

A performance measure is some metric by which we gauge how well our model is doing. Here I've chose to use a standard tool for the job: the root mean-squared error, or RMSE.

The mean-squared error takes the average distance between predicted values and actual values, and the root mean-squared error merely takes the square root of of the mean-squared error. The advantage of taking this final step  is that the units are preserved (i.e. 'dollars' as opposed to 'dollars squared'), and the metric is therefore more easily interpretible. 

Validating a time series model can be a little tricky. Here, I've divided the data into a 'train' set and a 'test' set, each comprising 50% of the total data, and elected to use a 'walk forward' test strategy. 

In essence, the model takes the entire training set and makes a single prediction with it, which it compares to the first actual data point in the test set. Then this data point is added to the training set -- which therefore grows in size by one data point -- and makes another single prediction, which is compared to the next data point in the test set. 

At each step in this walking process the difference between the predicted and actual values is noted and, when the process is finished, a final RMSE is calculated telling us how far off our predictions tended to be, on average. 

In [10]:
history = [x for x in train]
predictions = list()

# walk-forward validation 
for i in range(len(test)):
    model = ARIMA(history, order=(1,2,1))
    model_fit = model.fit(disp=0)
    yhat = model_fit.forecast()[0]
    predictions.append(yhat)
    obs = test[i]
    history.append(obs)
    if i % 10 == 0: # so as to not print out literally every iteration.
        print('>Predicted = %.3f, Expected = %.3f' % (yhat, obs))

# report performance
mse = mean_squared_error(test, predictions)
rmse = sqrt(mse)
print('RMSE: %.3f' % rmse)

>Predicted = 8163.322, Expected = 7838.611
>Predicted = 7231.575, Expected = 6674.394
>Predicted = 6629.826, Expected = 6532.896
>Predicted = 6024.098, Expected = 5985.887
>Predicted = 6334.930, Expected = 6404.785
>Predicted = 7613.237, Expected = 7555.662
>Predicted = 7008.918, Expected = 7238.580
>Predicted = 8172.398, Expected = 7960.392
>Predicted = 9102.172, Expected = 9232.296
>Predicted = 9244.526, Expected = 8774.818
>Predicted = 7955.099, Expected = 7682.265
>Predicted = 6882.474, Expected = 6815.312
>Predicted = 6933.993, Expected = 7773.527
>Predicted = 8488.701, Expected = 8046.967
>Predicted = 9056.909, Expected = 9117.040
>Predicted = 10375.357, Expected = 10115.465
>Predicted = 10912.562, Expected = 10006.713
RMSE: 355.370


I'm uncertain of the best way to interpret this RMSE, but given that the standard deviation for this data set is approximately [1] 2527.235:

In [11]:
mdbtc.describe()

,price_open
count,415.000000
mean,6993.243803
std,2527.232533
min,3237.278203
25%,6110.909430
50%,6566.161727
75%,8196.596443
max,16781.302024


 our model isn't doing too badly.
 
[1] - It might be a little more or a little less, because the std was calculated on the full `price_open` values for the market data, while the model worked on `train` and `test` data which were subsets of these market data. 

### Seasonal Autoregressive Integrated moving-average 

In [ ]:
model = SARIMAX(mdbtc_arr, order=(1, 1, 1), seasonal_order=(1, 1, 1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr))
print(yhat[0])

## Supervised Learning Predictive Model

[Machine Learning Mastery](https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/)

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
        Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[0]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

yhat = series_to_supervised(list(mdbtc_arr))
print(yhat)

### References

[Univariate predictive models (from Machine Learning Mastery)](https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/)  
[Building and Optimizing ARIMA models for time-series predictions](https://machinelearningmastery.com/time-series-forecast-case-study-python-monthly-armed-robberies-boston/)

### Future Directions

[LSTM predictions](https://lilianweng.github.io/lil-log/2017/07/08/predict-stock-prices-using-RNN-part-1.html)   
[Code for LSTM predictions](https://github.com/lilianweng/stock-rnn)  

[Doing the same with TF and GCP](https://medium.com/google-cloud/how-to-do-time-series-prediction-using-rnns-and-tensorflow-and-cloud-ml-engine-2ad2eeb189e8)  

[Echo State Networks](https://towardsdatascience.com/predicting-stock-prices-with-echo-state-networks-f910809d23d4)  